### Check All Tracks Match

In [1]:
# Imports
import numpy as np
import pandas as pd

In [2]:
# Read Files
my_uids = pd.read_csv('../Data/5_Features_with_ID_Full_Dataset.csv')
correct_uids = pd.read_csv('../Data/Results2_0_yolo_for_track_check.csv')

# Check Files
print(my_uids.tail())
print(correct_uids.tail())

        speed  outreach_ratio  mean_acceleration  distance_traveled  cluster  \
666  7.188265        0.222151           0.562417        4535.982335        3   
667  4.243637        0.212326           0.447011        1659.786611        3   
668  1.225791        0.143379           0.143933        5789.845947        6   
669  2.434414        0.081309           0.216220        4567.240841        6   
670  5.231545        0.056981           0.443116        1972.812041        3   

     is_male                                             id  
666        1  MC_singlenuc96_b1_Tk41_081120__0001_vid__8792  
667        1  MC_singlenuc96_b1_Tk41_081120__0001_vid__8893  
668        0  MC_singlenuc96_b1_Tk41_081120__0001_vid__9161  
669        0  MC_singlenuc96_b1_Tk41_081120__0001_vid__9345  
670        1  MC_singlenuc96_b1_Tk41_081120__0001_vid__9350  
        Unnamed: 0                          trial base_name  track_id  frame  \
112066      112066   MC_singlenuc81_1_Tk51_072920  0002_vid      12

In [3]:
# Find Unique Track Names from Correct
correct_uids['id'] = correct_uids['trial'] + '__' + correct_uids['base_name'] + '__' + correct_uids['track_id'].astype(str)
correct_uids['label'] = correct_uids['label'].map({'female': 0, 'male': 1})
correct_uids = correct_uids.drop(columns=['trial'])
correct_uids = correct_uids.drop(columns=['base_name'])
correct_uids = correct_uids.drop(columns=['track_id'])
correct_uids = correct_uids.drop(columns=['frame'])
correct_uids = correct_uids.drop(columns=['c_female'])
correct_uids = correct_uids.drop(columns=['c_male'])
correct_uids = correct_uids.drop(columns=['prediction'])
# correct_uids = correct_uids.drop(columns=['label'])
correct_uids = correct_uids.drop(columns=['acc'])
correct_uids = correct_uids.loc[:, ~correct_uids.columns.str.contains('^Unnamed')]
correct_uids = correct_uids.groupby('id')['label'].mean().reset_index()
correct_uids['label'] = correct_uids['label'].round()
correct_uids = pd.DataFrame(correct_uids)
print(correct_uids.tail())

                                                id  label
666  MC_singlenuc96_b1_Tk41_081120__0001_vid__8792    1.0
667  MC_singlenuc96_b1_Tk41_081120__0001_vid__8893    1.0
668  MC_singlenuc96_b1_Tk41_081120__0001_vid__9161    0.0
669  MC_singlenuc96_b1_Tk41_081120__0001_vid__9345    0.0
670  MC_singlenuc96_b1_Tk41_081120__0001_vid__9350    1.0


In [4]:
# Find Unique Track Names from Mine
my_uids['id'] = my_uids['id'].unique()
my_uids = pd.DataFrame(my_uids)
my_uids = my_uids.drop(columns=['speed'])
my_uids = my_uids.drop(columns=['mean_acceleration'])
my_uids = my_uids.drop(columns=['outreach_ratio'])
my_uids = my_uids.drop(columns=['distance_traveled'])
my_uids = my_uids.drop(columns=['cluster'])
print(my_uids.tail())

     is_male                                             id
666        1  MC_singlenuc96_b1_Tk41_081120__0001_vid__8792
667        1  MC_singlenuc96_b1_Tk41_081120__0001_vid__8893
668        0  MC_singlenuc96_b1_Tk41_081120__0001_vid__9161
669        0  MC_singlenuc96_b1_Tk41_081120__0001_vid__9345
670        1  MC_singlenuc96_b1_Tk41_081120__0001_vid__9350


In [5]:
# Check Matches
match = set(correct_uids['id']) == set(my_uids['id'])
print(f"Do all unique IDs in correct_uids match those in my_uids? {match}")

Do all unique IDs in correct_uids match those in my_uids? True


In [6]:
# Check Labels
merged_df = pd.merge(correct_uids, my_uids, on='id')
labels_match = (merged_df['label'] == merged_df['is_male']).all()
if labels_match:
    print("Label and is_male match for all rows after merge.")
else:
    print("Label and is_male do not match for all rows after merge.")

Label and is_male match for all rows after merge.


### Check Percentage of Frames Match (Abandoned)

In [7]:
# Read Files
my_uids = pd.read_csv('../Data/combined_dataset_for_uid_check.csv')
correct_uids = pd.read_csv('../Data/Results2_0_yolo_for_track_check.csv')

In [8]:
# Check Files
print(my_uids.tail())
print(correct_uids.tail())

                       experiment_name video_base_name  track_id  \
1001961  MC_singlenuc96_b1_Tk41_081120        0001_vid      9350   
1001962  MC_singlenuc96_b1_Tk41_081120        0001_vid      9350   
1001963  MC_singlenuc96_b1_Tk41_081120        0001_vid      9350   
1001964  MC_singlenuc96_b1_Tk41_081120        0001_vid      9350   
1001965  MC_singlenuc96_b1_Tk41_081120        0001_vid      9350   

         count_uncalibrated  new_count_uncalibrated          xc          yc  \
1001961                  95                  537123  156.276651  671.360475   
1001962                  96                  537124  162.206921  672.395285   
1001963                  97                  537125  170.841343  672.257588   
1001964                  98                  537126  178.862178  673.323482   
1001965                  99                  537127  187.945173  674.862680   

            u_dot     v_dot      s_dot  class_id   p_value  track_length  \
1001961  4.437002  6.482767  23.792231  

In [9]:
# Make UIDs for Both
my_uids['uid'] = my_uids['experiment_name'] + '__' + my_uids['video_base_name'] + '__' + my_uids['track_id'].astype(str)+ '__' + my_uids['frame'].astype(str)
correct_uids['uid'] = correct_uids['trial'] + '__' + correct_uids['base_name'] + '__' + correct_uids['track_id'].astype(str)+ '__' + correct_uids['frame'].astype(str)

In [10]:
# We Only Need UID
my_uids = my_uids['uid']
correct_uids = correct_uids['uid']
my_uids = pd.DataFrame(my_uids)
correct_uids = pd.DataFrame(correct_uids)

# Check
print(my_uids.tail())
print(correct_uids.tail())

                                                       uid
1001961  MC_singlenuc96_b1_Tk41_081120__0001_vid__9350_...
1001962  MC_singlenuc96_b1_Tk41_081120__0001_vid__9350_...
1001963  MC_singlenuc96_b1_Tk41_081120__0001_vid__9350_...
1001964  MC_singlenuc96_b1_Tk41_081120__0001_vid__9350_...
1001965  MC_singlenuc96_b1_Tk41_081120__0001_vid__9350_...
                                                      uid
112066  MC_singlenuc81_1_Tk51_072920__0002_vid__1220__...
112067    MC_singlenuc40_2_Tk3_030920__0001_vid__18__5840
112068  MC_singlenuc59_4_Tk61_060220__0001_vid__4920__...
112069  MC_singlenuc45_7_Tk47_050720__0002_vid__3133__452
112070  MC_singlenuc35_11_Tk61_051220__0002_vid__612__...
